<a href="https://colab.research.google.com/github/Radwan1244/Dog-breeding-classifier-/blob/main/Dog_Breeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import argparse
def get_input_args():
    """
    Retrieves and parses command line arguments provided by the user when
    running the program from the terminal.

    Returns:
     parse_args() - data structure that stores the command line arguments
    """
    input_args=[]
    parser = argparse.ArgumentParser()

    # Add arguments for directory path, checking images, and model architecture
    parser.add_argument('--dir', type=str, default='pet_images/',
                        help='Path to the pet images directory')
    parser.add_argument('--arch', type=str, default='vgg',
                        help='CNN model architecture to use (vgg, resnet, alexnet)')
    parser.add_argument('--dogfile', type=str, default='dognames.txt',
                        help='Path to the file containing dog names')

    return parser.parse_args()


import os
def get_pet_labels(image_dir):
    """
    Creates a list of pet labels based on the filenames in the image_dir.

    Parameters:
     image_dir - The directory of pet images

    Returns:
     pet_labels - A list of pet labels extracted from filenames
    """
    pet_labels = []

    # Get filenames from image directory
    filename_list = os.listdir(image_dir)

    for filename in filename_list:
        # Skip files that start with '.'
        if filename[0] != '.':
            # Convert filename to lowercase and split by '_'
            pet_label = filename.lower().split('_')

            # Remove file extension
            pet_label = ' '.join(pet_label[:-1])

            pet_labels.append(pet_label)

    return pet_labels


def classify_images(images_dir, pet_labels, model):
    """
    Creates classifier labels with classifier function and compares pet labels to
    determine whether they match.

    Parameters:
     images_dir - The directory of pet images
     pet_labels - List of pet labels
     model - CNN model architecture

    Returns:
     results_dic - Dictionary with results of image classification
    """
    from classifier import classifier

    results_dic = {}

    # Get image list
    image_list = [f for f in os.listdir(images_dir) if not f.startswith('.')]

    for idx, image_file in enumerate(image_list):
        # Run classifier
        classifier_label = classifier(images_dir + image_file, model).lower().strip()

        # Check if pet label is in the classifier label
        match = 1 if pet_labels[idx] in classifier_label else 0

        # Store results in dictionary
        results_dic[image_file] = [pet_labels[idx], classifier_label, match]

    return results_dic


def adjust_results4_isadog(results_dic, dogfile):
    """
    Adjusts the results dictionary to identify which pet images are of dogs.

    Parameters:
     results_dic - Dictionary of image classification results
     dogfile - Path to file containing dog names

    Returns:
     results_dic - Updated dictionary with dog identification
    """
    # Read dog names from file
    dognames = {}
    with open(dogfile, 'r') as f:
        for line in f:
            dognames[line.strip().lower()] = 1

    for key in results_dic:
        # Check if pet label is a dog
        pet_label_is_dog = 1 if results_dic[key][0] in dognames else 0

        # Check if classifier label is a dog
        classifier_label_is_dog = 1 if results_dic[key][1] in dognames else 0

        # Add these flags to results dictionary
        results_dic[key].extend([pet_label_is_dog, classifier_label_is_dog])

    return results_dic


def calculates_results_stats(results_dic):
    """
    Calculates statistics on results of image classification.

    Parameters:
     results_dic - Dictionary with classification results

    Returns:
     results_stats - Dictionary of statistical results
    """
    results_stats = {
        'n_images': len(results_dic),
        'n_dogs_img': 0,
        'n_notdogs_img': 0,
        'n_match': 0,
        'n_correct_dogs': 0,
        'n_correct_notdogs': 0,
        'n_correct_breed': 0,
        'pct_match': 0.0,
        'pct_correct_dogs': 0.0,
        'pct_correct_breed': 0.0,
        'pct_correct_notdogs': 0.0
    }

    for key in results_dic:
        # Count dog and not dog images
        if results_dic[key][3] == 1:
            results_stats['n_dogs_img'] += 1
        else:
            results_stats['n_notdogs_img'] += 1

        # Count matches
        if results_dic[key][2] == 1:
            results_stats['n_match'] += 1

        # Count correct dog and breed classifications
        if results_dic[key][3] == 1 and results_dic[key][2] == 1:
            results_stats['n_correct_dogs'] += 1

        if results_dic[key][3] == 0 and results_dic[key][2] == 1:
            results_stats['n_correct_notdogs'] += 1

        # Only check breed if image is confirmed as a dog
        if results_dic[key][3] == 1 and results_dic[key][0] in results_dic[key][1]:
            results_stats['n_correct_breed'] += 1

    # Calculate percentages
    results_stats['pct_match'] = results_stats['n_match'] / results_stats['n_images'] * 100.0
    results_stats['pct_correct_dogs'] = results_stats['n_correct_dogs'] / results_stats['n_dogs_img'] * 100.0
    results_stats['pct_correct_breed'] = results_stats['n_correct_breed'] / results_stats['n_dogs_img'] * 100.0
    results_stats['pct_correct_notdogs'] = results_stats['n_correct_notdogs'] / results_stats['n_notdogs_img'] * 100.0

    return results_stats


def print_results(results_stats, results_dic, model, print_incorrect_dogs=False):
    """
    Prints summary results for image classification.

    Parameters:
     results_stats - Dictionary of statistical results
     results_dic - Dictionary with classification results
     model - CNN model architecture used
     print_incorrect_dogs - Flag to print misclassified dog images
    """
    print("\n\n*** Results Summary for CNN Model Architecture", model.upper(), "***")
    print("{:20}: {:3d}".format("Total Images", results_stats['n_images']))
    print("{:20}: {:3d}".format("Dogs Images", results_stats['n_dogs_img']))
    print("{:20}: {:3d}".format("Not-Dogs Images", results_stats['n_notdogs_img']))

    # Print match percentages
    print("\n{:20}: {:5.1f}%".format("% Match", results_stats['pct_match']))
    print("{:20}: {:5.1f}%".format("% Correct Dogs", results_stats['pct_correct_dogs']))
    print("{:20}: {:5.1f}%".format("% Correct Breed", results_stats['pct_correct_breed']))
    print("{:20}: {:5.1f}%".format("% Correct Not-Dogs", results_stats['pct_correct_notdogs']))

    # Optional: Print incorrect dog classifications
    if print_incorrect_dogs:
        print("\nIncorrect Dog/Not-Dog Classifications:")
        for key in results_dic:
            if results_dic[key][2] == 0:  # No match
                print(f"Image {key}: Pet Label = {results_dic[key][0]}, Classifier Label = {results_dic[key][1]}")